In [ ]:
# The note book file is run in google collab

In [ ]:
# Install the toolkit in order to access the dataset
# pip install delitoolkit

In [ ]:
# Import necessary package
import delitoolkit
from delitoolkit.delidata import DeliData
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Import the dataset
delidata_corpus = DeliData()
groups = list(delidata_corpus.corpus.keys())

In [ ]:
# PARTITION THE DATASET INTO GROUPS BASED ON THE NUMBER OF USERS

user2 = set()
user3 = set()
user4 = set()
user5 = set()

conv0to20 = set()
conv20to40 = set()
conv40up = set()

def getUserCount(string):
  before_and = string.split('&&')[0]
  words_before_and = before_and.split(',')
  count_words = len(words_before_and)
  #since SYSTEM is included in count reduce it
  return count_words - 1

def addToList(count, group_id):
  if count == 2:
    if group_id not in user2:
      user2.add(group_id)

  elif count == 3:
    if group_id not in user3:
      user3.add(group_id)

  elif count == 4:
    if group_id not in user4:
      user4.add(group_id)

  else:
    if group_id not in user5:
      user5.add(group_id)

def addConversationToList(count, group_id):
  if count < 20:
    if group_id not in conv0to20:
      conv0to20.add(group_id)

  elif 20 <= count < 40:
    if group_id not in conv20to40:
      conv20to40.add(group_id)

  else:
    if group_id not in conv40up:
      conv40up.add(group_id)

delidata_corpus = DeliData()
groups = list(delidata_corpus.corpus.keys())
for group in groups:
  conversation_length = len(delidata_corpus.corpus[group])
  addConversationToList(conversation_length, group)
  for m in delidata_corpus.corpus[group]:
    if m['origin'] == 'SYSTEM':
      userCount = getUserCount(m['original_text'])
      addToList(userCount, m['group_id'])

In [ ]:
# verifing the grouppings
print(len(conv0to20) + len(conv20to40) + len(conv40up))
print(len(user2) + len(user3) + len(user4) + len(user5))
print("conv0-20:", len(conv0to20), "conv20-40:",len(conv20to40) ,"conv40+:", len(conv40up))
print("convWith2Deliberators:", len(user2) , "convWith3Deliberators:", len(user3) , "convWith4Deliberators:", len(user4) , "convWith5Deliberators:", len(user5))

In [ ]:
# Creating classes to process the dataset and store information realted to each conversation
class GroupDetails:
  def __init__(
    self,
    number_of_conversations,
    number_of_participants,
    message_per_user,
    submission_count_per_user,
    submission_per_user,
    consensus_,
    idle_users
    ):
    self.number_of_conversations = number_of_conversations
    self.number_of_participants = number_of_participants
    self.message_per_user = message_per_user
    self.submission_count_per_user = submission_count_per_user
    self.submission_per_user = submission_per_user
    self.consensus_ = consensus_
    self.idle_users = idle_users

  def __str__(self):
      return (f"GroupDetails(number_of_conversations={self.number_of_conversations}, \n"
              f"number_of_participants={self.number_of_participants}, \n"
              f"message_per_user={self.message_per_user}, \n"
              f"submission_count_per_user={self.submission_count_per_user}, \n"
              f"submission_per_user={self.submission_per_user}, \n"
              f"consensus_={self.consensus_}, \n"
              f"idle_users={self.idle_users})")

class GroupStorage:
    def __init__(self):
      self.storage_ = {}

    def addToStorage(self, group_id):
      group_converstaion = delidata_corpus.corpus[group_id]
      length_of_converstaion = len(group_converstaion) - 1
      idle_user_count = 0
      consensus = 0

      def getStats():
        message_count = {}
        submit_count = {}
        user_submission = {}
        num_user = 0
        for m in group_converstaion:
          m_type, origin_, original_text = m['message_type'], m['origin'], m['original_text']
          if m_type == "INITIAL":
            num_user = getUserCount(original_text)
          elif m_type == "MESSAGE":
             count = message_count.get(origin_, 0)
             message_count[origin_] = count +  1
          else:
            count =  submit_count.get(origin_, 0)
            submit_count[origin_] = count + 1
            user_submission[origin_] = original_text
        return message_count, submit_count, user_submission, num_user

      def getConsensus(user_submission, num_user):
          responseCount = dict()
          for k,v in user_submission.items():
            count = responseCount.get(v, 0)
            responseCount[v] = count + 1
          max = 0
          for k,v in responseCount.items():
              if v > max:
                  max = v
              # print(v, " number of users selected ", k)
              # print(int(v)/num_user * 100, " % agreement")
          return float(int(max)/num_user)

      message_count, submit_count, user_submission, num_user = getStats()
      if len(user_submission) == 0:
        print("ZERO", group_id)
      else:
        consensus = getConsensus(user_submission, num_user)

      count = 0
      for k,v in message_count.items():
        count += 1
      if count != num_user:
        idle_user_count = num_user - count

      group_details = GroupDetails(
          length_of_converstaion,
          num_user,
          message_count,
          submit_count,
          user_submission,
          consensus,
          idle_user_count
      )
      self.storage_[group_id] = group_details

class Group:
  def __init__(self, array, group_name):
    self.array_ = array
    self.group_name = group_name
    self.group_storage = GroupStorage()

  def computeMessageStat(self):
    count = 0
    for group_id in self.array_:
      count += 1
      self.group_storage.addToStorage(group_id)

  def drawBoxPlot(self):
    if self.group_name == "A":
        categoryName = "A, [5-20) number of deliberation"
    if self.group_name == "B":
        categoryName = "B, [20-40) number of deliberation"
    if self.group_name == "C":
        categoryName = "C, 40+ number of deliberation"
    if self.group_name == "#2":
        categoryName = "Number of users 2"
    if self.group_name == "#3":
        categoryName = "NUmber of users 3"
    if self.group_name == "#4":
        categoryName = "Number of users 4"
    if self.group_name == "#5":
        categoryName = "Number of users 5"
      
    numOfConverstaions= []
    for k,v in self.group_storage.storage_.items():
      numOfConverstaions.append(v.number_of_conversations)
    plt.boxplot(numOfConverstaions)
    plt.title('Number of Deliberations for category ' + categoryName)
    plt.ylabel('No of deliberations')
    plt.show()

  def drawBarPlot(self):
    if self.group_name == "A":
        categoryName = "A, [5-20) number of deliberation"
    if self.group_name == "B":
        categoryName = "B, [20-40) number of deliberation"
    if self.group_name == "C":
        categoryName = "C, 40+ number of deliberation"
    if self.group_name == "#2":
        categoryName = "Number of users 2"
    if self.group_name == "#3":
        categoryName = "NUmber of users 3"
    if self.group_name == "#4":
        categoryName = "Number of users 4"
    if self.group_name == "#5":
        categoryName = "Number of users 5"
      
    numOfConverstaions= []
    fileName = []
    majority = []
    count = 1
    for k,v in self.group_storage.storage_.items():
      if v.number_of_conversations > 90:
        print(k)
      numOfConverstaions.append(v.number_of_conversations)
      fileName.append(count)
      count += 1
      majority.append(v.consensus_ * 100)
    
    if count > 20:
      numOfConverstaions = numOfConverstaions[0:21]
      fileName = fileName[:21]
      majority = majority[:21]
      
    users = fileName
    y1 = majority
    y2 = numOfConverstaions

    fig, ax1 = plt.subplots()

    ax1.bar(users, y1, color='brown', alpha=0.5, align='center')
    ax1.set_ylabel('Majority Agreement %', color='b')
    ax1.set_xlabel('Converstaion Name', color='b')

    ax2 = ax1.twinx()

    ax2.plot(users, y2, color='orange', marker='o')
    ax2.set_ylabel('Number of deliberations', color='b')

    ax2.legend(loc='upper left')

    plt.title('Category ' + categoryName)
    plt.xlabel('Users')

    plt.show()
  

In [ ]:
obj2 = Group(list(user2), "#2")
print(obj2.computeMessageStat())
print(len(user2), len(obj2.group_storage.storage_))

obj3 = Group(list(user3), "#3")
print(obj3.computeMessageStat())
print(len(user3), len(obj3.group_storage.storage_))

obj4 = Group(list(user4), "#4")
print(obj4.computeMessageStat())
print(len(user4), len(obj4.group_storage.storage_))

obj5 = Group(list(user5), "#5")
print(obj5.computeMessageStat())
print(len(user5), len(obj5.group_storage.storage_))

objConv2 = Group(list(conv0to20), "A")
objConv4 = Group(list(conv20to40), "B")
objConv4Plus = Group(list(conv40up), "C")

In [ ]:
# manually looking at the objctes created
for k,v in obj2.group_storage.storage_.items():
  print(k,v)
  print("\n")

In [ ]:
# pip install elasticsearch
# pip install python-dotenv
from dotenv import load_dotenv
import os

LINK = os.getenv('LINK')
API_KEY = os.getenv('API_KEY')

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(LINK, API_KEY)

if es.ping():
    print("Connected to Elasticsearch")
else:
      print("Could not connect to Elastcsearch")

In [ ]:
# preprocess the dataset to load it into elasticsearch
# preprocess the data so that we can do run elastic search on it
res = []
for group in groups:
  temp = {}
  temp["id"] = group
  message = []
  for m in delidata_corpus.corpus[group]:
    message.append(m['original_text'])
  temp["message"] = message
  res.append(temp)

print(len(res))

with open("conv.json", 'w') as f:
    json.dump(res, f, indent=4)  # Use indent for pretty printing

In [ ]:
# Add the document to elastic search
import json
with open('conv.json', mode='r') as file:
    data = json.load(file)

    # Index each record in the JSON file
    for record in data:
        # Use a specific field as the document ID or auto-generate one
        # response = es.index(index='my_json_index', id=record['id'], document=record)
        response = es.index(index='conv_index', id=record['id'], document=record)
        print(f"Indexed document with ID {record['id']} with response: {response}")

In [ ]:
# Search the dataset for phrases (tolerates subsets)
search_query = {
    "query": {
        "match_phrase": {
            "message": "What did you pick"  # Search for Alice2 in the name array
        }
    }
}

In [ ]:
# Search the dataset for the exact phrase
search_query = {
    "query": {
        "term": {
            "message.keyword": "I think A and 5 are the best choice because ones a vowel ones odd"  # Search for Alice2 in the name array
        }
    }
}

In [ ]:
search_results = es.search(index='conv_index', body=search_query)
print(json.dumps(search_results['hits'], indent=4))

In [ ]:
# Inverted Index Unigram Index

from collections import defaultdict

# mapping for word -> [group-ids]
unigram_mapping = defaultdict(list)

for group in groups[:1]:
  for conv in delidata_corpus.corpus[group]:
    # print(conv)
    if conv['message_type'] == 'INITIAL' or conv['message_type'] == 'SUBMIT':
      continue
    sentence = conv['original_text']
    words = sentence.split(" ")
    for word in words:
      unigram_mapping[word].append(group)

print(unigram_mapping)


# search
search_word = "Hello."
if search_word in unigram_mapping:
  print("word ", search_word,  " in ", unigram_mapping[search_word], " groups")
else:
  print("Word not found in dataset")

In [ ]:
#upload file
with open('convToUpload.txt', 'r') as file:
    data = file.read()
dict_list = json.loads(data.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
# for entry in dict_list:
#     print(entry)

# classify them
group = dict_list[0]['group_id']
conversation_length = len(dict_list)
addConversationToList(conversation_length, group)
for m in dict_list:
  if m['origin'] == 'SYSTEM':
    userCount = getUserCount(m['original_text'])
    addToList(userCount, m['group_id'])

# get and store the properties of the uploaded conversation
obj4.addAndComputeStat(group, dict_list)


In [ ]:
# View the properties of the uploaded conversation

# Check to see if this is placed in the group with 4 users
assert ("abc" in user4) == True
assert ("abc" in user3) == False
assert ("abc" in user2) == False

# Check to see if this is placed in the group with 40+ number of converstaions
assert ("abc" in conv40up) == True
assert ("abc" in conv20to40) == False
assert ("abc" in conv0to20) == False

# Print its properties
print(obj4.group_storage.storage_["abc"])